In [2]:
%%bash
pip install --upgrade pip
pip install smart_open minecart
pip install textract-trp

In [3]:
import time 
import re
import os
import trp
import boto3
import minecart
import json
import logging 

import numpy as np
import pandas as pd

from smart_open import open
from sagemaker.session import Session

In [4]:
# initiate s3 bucket and corresponding data folder
bucket = "ran-s3-systemic-risk"
data_folder ="Input/X-17A-5-Subsets/"

# script to perform OCR (using Textract) for X-17A-5 subsets
out_folder = 'Output/X-17A-5-BS/'

# Amazon Textract client and Sagemaker session
textract = boto3.client('textract')
s3 = boto3.client('s3')
session = Session()

# AWS Asynchronous Textract Script (requesting Job)
**Content modified from Amazon AWS Textract repository (refer to [URL](https://github.com/aws-samples/amazon-textract-code-samples/blob/master/python/12-pdf-text.py) below)** 

In [5]:
def startJob(s3BucketName:str, objectName:str) -> str:
    """
    Starts a Textract job on AWS server 
    """
    response = None
    client = boto3.client('textract')
    
    # issue response to AWS to start Textract job for table analysis 
    response = client.start_document_analysis(
        DocumentLocation={
            'S3Object': {
                'Bucket': s3BucketName,
                'Name': objectName
            }
        },
        FeatureTypes=['FORMS']    # selecting forms from the OCR
    )
    
    # return response job ID for service
    return response["JobId"]

In [6]:
def isJobComplete(jobId:str) -> str:
    """
    Tracks the completion status of the Textract job when qued
    """
    time.sleep(1)
    client = boto3.client('textract')
    response = client.get_document_analysis(JobId=jobId)
    status = response["JobStatus"]
    print("Job status: {}".format(status))
    
    # check current status of AWS job (ask server every 5 seconds for data)
    while(status == "IN_PROGRESS"):
        time.sleep(5)                   # lag before reporting status
        response = client.get_document_analysis(JobId=jobId)
        status = response["JobStatus"]
        print("Job status: {}".format(status))
    
    return status

In [7]:
def getJobResults(jobId:str) -> list:
    """
    Returns the contents of the Textract job, after completion status met
    """
    pages = []          # initialize list object to track pages

    client = boto3.client('textract')
    response = client.get_document_analysis(JobId=jobId)
    
    pages.append(response)
    print("Resultset page recieved: {}".format(len(pages)))
    
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']
    
    # iterate through the pages and append to response figure
    while(nextToken):
        response = client.get_document_analysis(JobId=jobId, NextToken=nextToken)
        pages.append(response)
        print("Resultset page recieved: {}".format(len(pages)))
        nextToken = None
        if('NextToken' in response):
            nextToken = response['NextToken']

    return pages

In [8]:
def runJob(bucket:str, key:str):
    """
    Function designed to call an AWS Textract 
    """
    # S3 storage for files on AWS site   
    jobId = startJob(bucket, key)   # intialize Textract job 
    print("Started job with id: {}".format(jobId))

    # if job is complete from AWS return response object 
    if(isJobComplete(jobId)):
        response = getJobResults(jobId)
        
    return response

In [9]:
# temporary data frame object for balance sheet information
res1 = runJob("ran-s3-systemic-risk", 'Input/X-17A-5-Subsets/42352-2005-subset.pdf')

Started job with id: 6b7e541c60eedcbaa731b2aac4cf80a9eb22312d87b234a0620ed6895ad18c0f
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2


In [72]:
# format the Textract response type 
doc = trp.Document(res1)

key_map = {}
value_map = {}
block_map = {}
    
# iterate through document pages
for i, page in enumerate(doc.pages):

    # itterate through page tables
    for j, block in enumerate(page.blocks): 
        block_id = block['Id']
        block_map[block_id] = block
        if block['BlockType'] == "KEY_VALUE_SET":
            if 'KEY' in block['EntityTypes']:
                key_map[block_id] = block
            else:
                value_map[block_id] = block


In [73]:
def get_kv_relationship(key_map, value_map, block_map):
    kvs = {}
    for block_id, key_block in key_map.items():
        value_block = find_value_block(key_block, value_map)
        key = get_text(key_block, block_map)
        val = get_text(value_block, block_map)
        kvs[key] = val
    return kvs


def find_value_block(key_block, value_map):
    for relationship in key_block['Relationships']:
        if relationship['Type'] == 'VALUE':
            for value_id in relationship['Ids']:
                value_block = value_map[value_id]
    return value_block


def get_text(result, blocks_map):
    text = ''
    if 'Relationships' in result:
        for relationship in result['Relationships']:
            if relationship['Type'] == 'CHILD':
                for child_id in relationship['Ids']:
                    word = blocks_map[child_id]
                    if word['BlockType'] == 'WORD':
                        text += word['Text'] + ' '
                    if word['BlockType'] == 'SELECTION_ELEMENT':
                        if word['SelectionStatus'] == 'SELECTED':
                            text += 'X '    

                                
    return text

In [74]:
get_kv_relationship(key_map, value_map, block_map)

{'(City) ': 'New York ',
 '(Zip Code) ': '10036 ',
 'MM/DD/YY ': '11/26/04 ',
 '(State) ': 'New York ',
 'Public Accountant ': '',
 'OMB Number: ': '3235-0123 ',
 'Accountant not resident in United States or any of its possessions. ': '',
 'Certified Public Accountant ': 'X ',
 '(Area Code -Telephone No.) ': '(212)357-8710 ',
 'Expires: ': '',
 'hours per response ': '12.00 ',
 '(Address) ': '1177 Avenue of the Americas ',
 'FIRM ID NO. ': '13-5108880 ',
 'FOR OFFICIAL USE ONLY ': '',
 'SEC FILE NUMBER ': '8- 129 ',
 '(Name - if individual, state last, first, middle name) ': 'PricewaterhouseCoopers LLP ',
 'NAME OF BROKER-DEALER: ': 'Goldman, Sachs & Co. ',
 'Sarah Smith Managing Director ': '',
 'David A. Viniar Managing Director ': 'J.N. ',
 'Subscribed and sworn before me; ': 'L. Kelly Samands ',
 'No. ': '2176936 ',
 '(Signature) ': 'Jus ',
 '(Type Name and Title) ': 'David A. Viniar, Managing Director ',
 "Total liabilities and partners' capital ": '$ 348,590,977 ',
 'Securities l

# OCR Wrapper Functions
**The scripts perform an OCR job from AWS Textract, converting tabular data into dataframes**

In [7]:
def trp2df(table:trp.Table) -> pd.DataFrame:
    """
    Function designed to convert a trp table into a dataframe
    :param table: a trp table object parsed from a pdf  
    :return: a DataFrame object housing a textracted trp table
    
    Complexity -> O(n^2) approx.
    """
    N = len(table.rows)               # number of rows in table
    M = len(table.rows[0].cells)      # number of columns in table
    arr = [0]*N
    
    # iterate through each row within the provided table
    for row in np.arange(N):
        
        # strip the text from the cell references to construct (N X M) matrix
        arr[row] = [table.rows[row].cells[col].text.strip() for col in np.arange(M)]
        
    return pd.DataFrame(arr)

In [13]:
def readPDF(response:list) -> pd.DataFrame:
    """
    Function to transform AWS Textract object to a pdf
    :param response: AWS Textract response object
    """
    # in the event multiple tables detected on one page (concat them)
    catDF = []
    
    # format the Textract response type 
    doc = trp.Document(response)
    
    # iterate through document pages
    for page in doc.pages:
        
        # itterate through page tables
        for table in page.tables: 
            
            # convert trp-table into dataframe object
            df = trp2df(table)
            
            # remove columns that are completly empty
            empty_cols = [col for col in df.columns if (df[col] == '').all()]
            df = df.drop(empty_cols, axis=1)
  
            # number of columns in dataframe
            n = df.columns.size
            
            # reset the column names (avoid the column names)
            df.columns = np.arange(n)
            
            ##############################################################
            #                           NOTES
            #          a good dataframe should have 2-3 columns
            #      anything more or less is a reading error we ignore
            ##############################################################
            
            # if the dataframe has more than 3 columns then we most likley have an issue in parsing
            if n > 3:
                return None
            
            elif n > 1:
                
                ##############################
                # Balance Sheet Assummptions
                ##############################
                
                lineIndex = df.columns[0]

                # check for the word "cash" in a string at the begining, ignoring case sensitivity (asset check)
                assetCheck = df[lineIndex].str.contains('^Cash', regex=True, flags=re.IGNORECASE)

                # check for the word "Liabilities" in a string at the end, ignoring case sensitivity (liability check)
                debtCheck1 = df[lineIndex].str.contains('Liabilities$|^Liabilities', regex=True, flags=re.IGNORECASE)
                debtCheck2 = df[lineIndex].str.contains('Liability$|^Liability', regex=True, flags=re.IGNORECASE)
                
                # check for the presence of $ sign, we assume the balance sheet items should have presence of $ sign
                dollarCheck = df[df.columns[1]].str.contains('\$[^\]]+', regex=True, flags=re.IGNORECASE)
                
                ##############################
                # Balance Sheet Determination
                ##############################
                
                # check if the key words have been found 
                check1 = df[assetCheck | debtCheck1 | debtCheck2].empty      # check for terms, and $ presence
                check2 = df[dollarCheck == True].empty                       # check for presence of '$' sign  
                check3 = df[debtCheck1 == True].empty                        # debt check for Liabilities
                check4 = df[debtCheck2 == True].empty                        # debt check for Liability 
                
                # if either asset term or liability term is found, with a $ sign we append the dataframe
                if not check1 and not check2:
                    catDF.append(df)      # we append since sometimes asset and liablility tables are seperated 

                    if not check3 or not check4:
                        # if liability table was found on the first iteration we simply concat data frames and return 
                        return pd.concat(catDF)
        

## Extract Balance Sheet information

In [9]:
def textractParse(path:str, index:int, csvDirectory:np.ndarray, bucket:str = "ran-s3-systemic-risk", 
                  out_folder:str = 'Output/X-17A-5-BS/') -> dict:
    """
    Function runs a Textract job and saves Balance Sheet information to .csv file in s3 folder 
    """
    errors = ''
    
    # baseFile name to name export .csv file e.g. 782124-2002.csv
    baseFile = '-'.join(path.split('/')[-1].split('-')[:2])
    fileName = baseFile + '.csv'
    print('\nPerforming OCR for {}'.format(baseFile))

    # if file is not found in directory we continue the iteration process
    if out_folder + fileName not in csvDirectory:

        # temporary data frame object for balance sheet information
        res = runJob("ran-s3-systemic-risk", path)
        
        # if Textract job did not fail we continue extraction
        if res[0]['JobStatus'] != 'FAILED':
            tempDF = readPDF(res)
            print(tempDF)
            
            # checks for type of return, if none then we log an error
            if type(tempDF) == pd.DataFrame:
                
                # writing data frame to .csv file
                tempDF.to_csv(fileName, index=False)

                # save contents to AWS S3 bucket
                with open(fileName, 'rb') as data:
                    s3.put_object(Bucket=bucket, Key=out_folder + fileName, Body=data)

                # remove local file after it has been created
                os.remove(fileName)
                
                print('-----------------------------------------------------')
                print('Saved {} file to s3 bucket'.format(baseFile + '.csv'))
            else:
                errors = 'No Balance Sheet found, or parsing error'
        else:
            errors = 'Could not parse, JOB FAILEDs'
    else:
        print('{} has been downloaded'.format(fileName))
        
    return errors

In [14]:
# csv Directory to store balance sheet information 
csvs = np.array(session.list_s3_files(bucket, out_folder))

# discover all of the pdfs that you want to parse
paths = np.array(session.list_s3_files(bucket, data_folder))[1:]

errorDict = {}

# iterate through X-17A-5 subsets stored in s3 
for i, key in enumerate(paths):     
    val = textractParse(key, i, csvs)
    
    if val != '':
        errorDict[key] = val


Performing OCR for 1224385-2004
1224385-2004.csv has been downloaded

Performing OCR for 1224385-2005
1224385-2005.csv has been downloaded

Performing OCR for 1224385-2006
1224385-2006.csv has been downloaded

Performing OCR for 1224385-2007
1224385-2007.csv has been downloaded

Performing OCR for 1224385-2008
1224385-2008.csv has been downloaded

Performing OCR for 1224385-2009
1224385-2009.csv has been downloaded

Performing OCR for 1224385-2010
1224385-2010.csv has been downloaded

Performing OCR for 1224385-2011
1224385-2011.csv has been downloaded

Performing OCR for 1224385-2012
1224385-2012.csv has been downloaded

Performing OCR for 1224385-2013
1224385-2013.csv has been downloaded

Performing OCR for 1224385-2014
1224385-2014.csv has been downloaded

Performing OCR for 1224385-2015
1224385-2015.csv has been downloaded

Performing OCR for 1224385-2016
1224385-2016.csv has been downloaded

Performing OCR for 1224385-2017
1224385-2017.csv has been downloaded

Performing OCR for 

Started job with id: 82d34a5d06d026e878cb42c467dbd7122955aea1edc38c81cdaf6a1cd146526f
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
Resultset page recieved: 6
None

Performing OCR for 851376-2008
851376-2008.csv has been downloaded

Performing OCR for 851376-2009
851376-2009.csv has been downloaded

Performing OCR for 851376-2010
851376-2010.csv has been downloaded

Performing OCR for 851376-2011
851376-2011.csv has been downloaded

Performing OCR for 851376-2012
851376-2012.csv has been downloaded

Performing OCR for 851376-2013
851376-2013.csv has been downloaded

Performing OCR for 851376-2014
851376-2014.csv has b

Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
Resultset page recieved: 6
Resultset page recieved: 7
                                                    0         1         2
0                                             Assets:                    
1                           Cash and cash equivalents               $ 739
2   Cash and recurities segregated and on deposit ...               1,914
3     Collateralized short-term financing agreements:                    
4     Securities purchased under agreements to resell   $64,019          
5               Deposits paid for securities borrowed    43,025          
6   Financial instruments owned and contractual co...             107,044
7   (Approximately $29 billion were pledged to var...                    
8    U.S. government and government agency securities    50,620          
9                           Corporate debt securities    16,080          
10                                  Equity securiti

In [15]:
# storing unique list of asset items and liability line items
with open('textractErrors.json', 'w') as f: json.dump(errorDict, f)